In [4]:
from geoband.API import *
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import geopandas as gpd
import pandas as pd
import json
from pandas import json_normalize
import pandas as pd
from geoband.API import *
import geopandas as gpd
import warnings
import matplotlib
from datetime import datetime 


# 1. 콤파스 [시민이 공감하는 치안 체감안전도 예측 및 분석] 데이터

In [19]:
GetCompasData('SBJ_2107_004', '11', '11.5개시도_교통사고현황.csv')
GetCompasData('SBJ_2107_004', '12', '12.경찰서별_지구대별_인원현황.csv')
GetCompasData('SBJ_2107_004', '4', '4.체감안전도_통계(2017~2019).csv')
GetCompasData('SBJ_2107_004', '6', '6.범죄발생원표.csv')
GetCompasData('SBJ_2107_004', '7', '7.범죄검거원표.csv')
GetCompasData('SBJ_2107_004', '8', '8.범죄피의자원표.csv')
GetCompasData('SBJ_2107_004', '19', '19.5개시도_최종학력통계.csv')

# 기타 참고 링크 : https://compas.lh.or.kr/subj/past/data?subjNo=SBJ_2107_004

In [20]:
car_accident=pd.read_csv('11.5개시도_교통사고현황.csv')
police_num=pd.read_csv('12.경찰서별_지구대별_인원현황.csv')
safe_feeling=pd.read_csv('4.체감안전도_통계(2017~2019).csv')
crime_commit=pd.read_csv('6.범죄발생원표.csv')
crime_caught=pd.read_csv('7.범죄검거원표.csv')
crimial=pd.read_csv('8.범죄피의자원표.csv')
latest_school=pd.read_csv('19.5개시도_최종학력통계.csv')

## 1. 교통사고 데이터 : jur_stn
- **경찰서별**비교 가능

|칼럼|칼럼명|
|:---|:---|
|date|사고날짜|
|acc_case|사고유형|
|ofn_sx|가해운전자 성별|
|ofn_age|가해운전자 연령|
|vic_sx|피해운전자 성별|
|vic_age|피해운전자 연령|
|jur_stn|관할서|

In [15]:
car_accident

,date,acc_case,ofn_sx,ofn_age,vic_sx,vic_age,jur_stn
0,2018년 1월,차대차 - 측면충돌,남,20대,남,50대,서울중부
1,2018년 1월,차대차 - 측면충돌,여,50대,남,70대,서울중부
2,2018년 1월,차대차 - 측면충돌,남,40대,남,50대,서울중부
3,2018년 1월,차대차 - 기타,기타불명,미분류,여,40대,서울중부
4,2018년 1월,차대차 - 기타,남,60대,남,50대,서울중부
...,...,...,...,...,...,...,...
140949,2019년 8월,차대사람 - 횡단중,남,40대,여,90대 이상,마산중부
140950,2020년 12월,차대사람 - 차도통행중,여,40대,여,90대 이상,창원중부
140951,2019년 4월,차대사람 - 횡단중,남,60대,남,90대 이상,진해경찰서
140952,2019년 5월,차대사람 - 차도통행중,남,20대,남,90대 이상,진해경찰서


## 2. 경찰서별_지구대별_인원현황 : pol_office
- **경찰서별** 비교 가능

|칼럼|칼럼명|
|:---|:---|
|nat_pol_agency|소속 경찰청|
|pol_office|소속 경찰서|
|sec_center|소속 치안센터|
|num|소속 되어 있는 총 원|


In [16]:
police_num

,nat_pol_agency,pol_office,sec_center,num
0,서울경찰청,서울중부경찰서,을지지구대,42
1,서울경찰청,서울중부경찰서,광희지구대,47
2,서울경찰청,서울중부경찰서,약수지구대,42
3,서울경찰청,서울중부경찰서,신당파출소,26
4,서울경찰청,서울중부경찰서,장충파출소,26
...,...,...,...,...
315,경남경찰청,진주경찰서,일반성파출소,13
316,경남경찰청,진주경찰서,지수파출소,9
317,경남경찰청,진주경찰서,수곡파출소,9
318,경남경찰청,진주경찰서,대곡파출소,9


## 3. 체감안전도_통계(2017~2019)
- **경찰서별, 연령대별, 성별**

In [27]:
safe_feeling

,large_category,medium_category,tot_f_safety,dept_safety,crime_safety,rob_satety_q1,mur_safety_q2,ta_safety_q3,raw_odder_q4,overall_q5
0,2017 상반기(연령대),10대(만14세 이상),74.3,73.3,81.5,80.1,82.8,70.9,66.9,76.5
1,2017 상반기(연령대),20대,70.1,69.3,76.9,75.7,78.2,66.1,64.3,71.7
2,2017 상반기(연령대),30대,69.6,68.9,76.8,75.7,77.9,63.9,65.2,71.1
3,2017 상반기(연령대),40대,70.7,70.2,77.2,76.1,78.3,65.5,67.0,71.9
4,2017 상반기(연령대),50대,72.1,71.5,78.3,77.7,79.0,68.0,67.6,73.3
...,...,...,...,...,...,...,...,...,...,...
285,2019 하반기(경찰서별),경남창원서부서,74.0,73.2,79.3,79.1,79.5,68.5,71.2,76.0
286,2019 하반기(경찰서별),경남마산중부서,75.4,74.9,80.4,80.8,80.0,70.8,73.2,76.5
287,2019 하반기(경찰서별),경남마산동부서,71.9,71.4,75.8,75.0,76.5,69.4,68.9,73.1
288,2019 하반기(경찰서별),경남진주서,75.1,74.9,79.2,78.7,79.6,70.3,74.6,75.6


## 4. 범죄발생원표
- **경찰서별** 비교 가능

In [22]:
crime_commit

,jur_stn,crm,crm_wthr,crm_clue,vic_sx,vic_age,crm_loc,crm_tm,crm_date,5m_crm_yn
0,서울수서경찰서,위조외국통화행사,미상,진정,불상,,은행,09:00-11:59,20170101,
1,서울영등포경찰서,도로교통법위반,맑음,타인신고,불상,,노상,21:00-23:59,20170101,
2,서울양천경찰서,209015100,미상,피해자신고,남자,60세초과,노상,미상,20170101,
3,서울서초경찰서,폭행,미상,피해자신고,여자,40세이하,기타,21:00-23:59,20170101,폭력
4,서울동대문경찰서,사기,미상,진정,여자,30세이하,기타,미상,20170101,
...,...,...,...,...,...,...,...,...,...,...
1073217,경남진해경찰서,사기,미상,고소,2,60세이하,기타,미상,9/27/2019,
1073218,경남진해경찰서,폭행,맑음,피해자신고,3,,차안,00:00∼02:59,10/4/2019,폭력
1073219,경남마산동부경찰서,재물손괴,미상,피해자신고,2,60세이하,주택,12:00∼14:59,9/17/2019,폭력
1073220,경남마산중부경찰서,사기,미상,진정,1,50세이하,기타,12:00∼14:59,9/14/2019,


## 5. 범죄검거원표
- **경찰서별** 비교 가능

In [23]:
crime_caught

,date,jur_stn,crm
0,201701,경남마산중부경찰서,사기
1,201701,경남마산중부경찰서,경범죄처벌법위반
2,201612,경남마산중부경찰서,도로교통법위반(음주운전)
3,201702,경남마산중부경찰서,위계공무집행방해
4,201702,경남마산중부경찰서,위계공무집행방해
...,...,...,...
928560,201912,경남창원중부경찰서,교통사고처리특례법위반(치상)
928561,201912,경남창원중부경찰서,사기
928562,201912,경남창원중부경찰서,사기
928563,201912,경남창원중부경찰서,사기


## 6. 범죄피의자원표
- **경찰서별** 비교 가능

In [24]:
crimial

,jur_stn,date,crm,ofn_age,ofn_sx,ofn_edu,ofn_sol
0,서울서초경찰서,201709,의료법위반,58,여,NaN,NaN
1,서울은평경찰서,201710,대외무역법위반,0,불상,NaN,NaN
2,서울은평경찰서,201710,대외무역법위반,52,여,대학졸업,상류
3,서울은평경찰서,201710,대외무역법위반,56,남,대학졸업,상류
4,서울남대문경찰서,201701,사기,59,여,미상,하류
...,...,...,...,...,...,...,...
1290331,경남마산동부경찰서,201908,사기,21,남,고등학교졸업,하류
1290332,경남마산동부경찰서,201906,사기,0,불상,NaN,NaN
1290333,경남마산중부경찰서,201906,사기,26,남,고등학교졸업,하류
1290334,경남마산중부경찰서,201910,저작권법위반,48,남,NaN,NaN


## 7. 5개시도_최종학력통계
- **구별** 비교 가능

In [25]:
latest_school

,sido,sgg_nm,emd_nm,sx,age,inv_edu,inv_detail,num_in_2015
0,서울특별시,종로구,사직동,남자,합계,계,6세 이상 내국인,3669.0
1,서울특별시,종로구,사직동,남자,합계,초등학교,6세 이상 내국인,339.0
2,서울특별시,종로구,사직동,남자,합계,중학교,6세 이상 내국인,237.0
3,서울특별시,종로구,사직동,남자,합계,고등학교,6세 이상 내국인,629.0
4,서울특별시,종로구,사직동,남자,합계,"대학교(2,3년제)",6세 이상 내국인,196.0
...,...,...,...,...,...,...,...,...
80339,경상남도,창원시진해구,웅동2동,여자,70세 이상,고등학교,6세 이상 내국인,57.0
80340,경상남도,창원시진해구,웅동2동,여자,70세 이상,"대학교(2,3년제)",6세 이상 내국인,8.0
80341,경상남도,창원시진해구,웅동2동,여자,70세 이상,대학교(4년제 이상),6세 이상 내국인,NaN
80342,경상남도,창원시진해구,웅동2동,여자,70세 이상,대학원(석박사 과정),6세 이상 내국인,NaN


# 2. 스마트 치안 빅데이터 플랫폼
- 여기 체감안전도(1.에 이미 있는) 밖에 활용 못할듯

# 3. PEST 관련

## Political
## Economical
|이름|분류|링크/장소|
|:--|:--|:--|
|GDP|도시(서울, 창원- 2013)|https://kosis.kr/statHtml/statHtml.do?orgId=101&tblId=DT_2KAA912 |
|경제성장률|전국(시도)|https://kosis.kr/statHtml/statHtml.do?orgId=101&tblId=DT_1YL20571&conn_path=I2 |
|실업률|시군구|https://kosis.kr/statHtml/statHtml.do?orgId=101&tblId=INH_1ES3A01S&vw_cd=MT_GTITLE01&list_id=106&seqNo=&
lang_mode=ko&language=kor&obj_var_id=&itm_id=&conn_path=MT_GTITLE01 |
|경제활동참가율|시군구|https://kosis.kr/search/search.do?query=%EC%8B%A4%EC%97%85%EB%A5%A0|
|고용률|시군구|https://kosis.kr/search/search.do?query=%EC%8B%A4%EC%97%85%EB%A5%A0|
|상용직 비중|시군구|https://kosis.kr/search/search.do?query=%EC%8B%A4%EC%97%85%EB%A5%A0|
|실업률|시군구|https://kosis.kr/search/search.do?query=%EC%8B%A4%EC%97%85%EB%A5%A0|
|실업자수|시군구|https://kosis.kr/search/search.do?query=%EC%8B%A4%EC%97%85%EB%A5%A0|
|청년고용률|시군구|https://kosis.kr/search/search.do?query=%EC%8B%A4%EC%97%85%EB%A5%A0|
|취업자수|시군구|https://kosis.kr/search/search.do?query=%EC%8B%A4%EC%97%85%EB%A5%A0|
|타지역 통근 취업자|시군구|https://kosis.kr/search/search.do?query=%EC%8B%A4%EC%97%85%EB%A5%A0|
|물가지수|시도|https://kosis.kr/|



### 제공 데이터
|이름|분류|
|:--|:--|


## Social-cultural
|이름|분류|링크/장소|
|:--|:--|:--|
|분위|시군구|https://kosis.kr/statHtml/statHtml.do?orgId=350&tblId=DT_35003_A0790&vw_cd=MT_ZTITLE&list_id=350_35003_4_1&seqNo=&
lang_mode=ko&language=kor&obj_var_id=&itm_id=&conn_path=MT_ZTITLE(타 시도 있음)|
|인구 10만명당 사회복지시설수|시군구|https://kosis.kr/statHtml/statHtml.do?orgId=101&tblId=DT_1YL20941&vw_cd=MT_GTITLE01&list_id=112&seqNo=&
lang_mode=ko&language=kor&obj_var_id=&itm_id=&conn_path=MT_GTITLE01|
|건강보험적용인구현황|시군구|https://kosis.kr/|
|노인 천명당 노인여가복지시설수|시군구|https://kosis.kr/|
|등록장애인수|시군구|https://kosis.kr/|
|보건및사회복지 사업체비율|시군구|https://kosis.kr/|
|보건및사회복지사업 종사자비율|시군구|https://kosis.kr/|
|일반회계중 사회복지예산비중|시군구|https://kosis.kr/|
|일반회계중 일반공공행정예산비중|시군구|https://kosis.kr/|
||||

### 제공 데이터
|이름|분류|
|:--|:--|
|수원시|시간대별_유동인구(2020)|
|수원시|성연령별_유동인구(2020)|
|수원시|요일별_유동인구(2020)|
|4개시도|주택노후도|
|4개시도|유흥업소현황|
|4개시도|용도지역도|
|4개시도|사업지구경계도|
|수원시|노숙인자활시설현황|
|수원시|아동복지시설현황|
|수원시|지역아동센터시설현황|
|수원시|청소년공부방현황|
|수원시|도서관현황|
|수원시|무료급식소현황|
|수원시|아동복지급식소현황|
|수원시|상권(업종)현황|
|수원시|건축물대장(표제부)|
|4개시도|공원현황|
|수원시|성연령별_거주인구격자(250M_X_250M)|
|4개시도|1인가구수|
|4개시도|기초수급자현황|
|4개시도|성연령별_총인구분포|
|4개시도|인구데이터(외국인)|
|4개시도|자살율|
|4개시도|화재발생통계|
|4개시도|어린이놀이시설현황|
|4개시도|토지이용계획도|
|4개시도|편의점현황|

## Technological : 데이터 딱히 없음...
|이름|분류|링크/장소|
|:--|:--|:--|

### 제공 데이터
|이름|분류|
|:--|:--|



### 제공 데이터 - 치안 직접 관련
|이름|분류|
|:--|:--|
|수원시|여성안심택배함현황|
|수원시|가로등현황|
|4개시도|보안등현황|
|4개시도|안전비상벨현황|
|4개시도|CCTV설치현황|


# 생각이 든 것
## <u>범죄가 많이 발생할만한 곳</u>을 예측하는 대회가 아니다
### -> 다른 시도와 차별되는 <u>수원시의 치안 특성</u>을 도출하는 대회이다
## 데이터를 많이 뽑아서 피쳐 선택을 한다기 보다는, 수원시의 치안 특성에 대해서 도출할 수 있는 방향을 선택해야한다.
### -> (전체 비교)정답을 수원, 서울, 진주, 창원으로 둔 뒤 디시전트리/랜덤포레스트?
- 경찰청 데이터가 포함된 데이터에 대하여 위 두 개 머신러닝 진행해서 시각화해서 결과 보기?
- 경찰서 관할 경계 단위로 진행해야하니까 경찰서별로 봐도 됨.

### -> (수원시 할 때)수원과 그 외로 위와 동일?
